#### 1. Выручка. Для каждого дня посчитать следующие показатели:
* Выручку, полученную в этот день.
* Суммарную выручку на текущий день.
* Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.

```sql
SELECT date,
       revenue,
       sum(revenue) OVER (ORDER BY date) as total_revenue,
       round(100 * (revenue - lag(revenue, 1) OVER (ORDER BY date))::decimal / lag(revenue, 1) OVER (ORDER BY date),
             2) as revenue_change
FROM   (SELECT creation_time::date as date,
               sum(price) as revenue
        FROM   (SELECT creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using (product_id)
        GROUP BY date) t2
```

#### 2. На основе данных о выручке рассчитаем несколько относительных показателей, которые покажут, сколько в среднем потребители готовы платить за услуги нашего сервиса доставки.
Посчитать:
* Выручку на пользователя (ARPU) за текущий день.
* Выручку на платящего пользователя (ARPPU) за текущий день.
* Выручку с заказа, или средний чек (AOV) за текущий день.

```sql
SELECT date,
       round(revenue::decimal / users, 2) as arpu,
       round(revenue::decimal / paying_users, 2) as arppu,
       round(revenue::decimal / orders, 2) as aov
FROM   (SELECT creation_time::date as date,
               count(distinct order_id) as orders,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using(product_id)
        GROUP BY date) t2
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as users
               FROM   user_actions
               GROUP BY date) t3 using (date)
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t4 using (date)
ORDER BY date
```

#### 3. Вычислить все те же метрики, но для каждого дня учитывать накопленную выручку и все имеющиеся на текущий момент данные о числе пользователей и заказов. 
Посчитать: 
* Накопленную выручку на пользователя (Running ARPU).
* Накопленную выручку на платящего пользователя (Running ARPPU).
* Накопленную выручку с заказа, или средний чек (Running AOV).

```sql
SELECT date,
       round(sum(revenue) OVER(ORDER BY date) / sum(new_users) OVER(ORDER BY date),
             2) as running_arpu,
       round(sum(revenue) OVER(ORDER BY date) / sum(new_paying_users) OVER(ORDER BY date),
             2) as running_arppu,
       round(sum(revenue) OVER(ORDER BY date) / sum(orders) OVER(ORDER BY date),
             2) as running_aov
FROM   (SELECT date,
               count(distinct order_id) as orders,
               sum(price) as revenue
        FROM   (SELECT date(creation_time) as date,
                       order_id,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in(SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')) as l
            LEFT JOIN products as r using(product_id)
        GROUP BY date) as revenue_by_date
    LEFT JOIN (SELECT date,
                      count(distinct user_id) filter(WHERE first_date = date) as new_users
               FROM   (SELECT date(time) as date,
                              user_id,
                              order_id,
                              min(date(time)) OVER(PARTITION BY user_id) as first_date
                       FROM   user_actions) as new_users
               GROUP BY date) as users1 using(date)
    LEFT JOIN (SELECT date,
                      count(distinct user_id) filter(WHERE first_date = date) as new_paying_users
               FROM   (SELECT date(time) as date,
                              user_id,
                              min(date(time)) OVER(PARTITION BY user_id) as first_date
                       FROM   user_actions
                       WHERE  order_id not in(SELECT order_id
                                              FROM   user_actions
                                              WHERE  action = 'cancel_order')) as new_paying_users
               GROUP BY date) as users2 using(date)
ORDER BY date
```

#### 4. Посчитать эти же показатели не по дням, а по дням недели за период с 26 августа 2022 года по 8 сентября 2022 года включительно.

```sql
SELECT weekday,
       t1.weekday_number as weekday_number,
       round(revenue::decimal / users, 2) as arpu,
       round(revenue::decimal / paying_users, 2) as arppu,
       round(revenue::decimal / orders, 2) as aov
FROM   (SELECT to_char(creation_time, 'Day') as weekday,
               max(date_part('isodow', creation_time)) as weekday_number,
               count(distinct order_id) as orders,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')
                   and creation_time >= '2022-08-26'
                   and creation_time < '2022-09-09') t4
            LEFT JOIN products using(product_id)
        GROUP BY weekday) t1
    LEFT JOIN (SELECT to_char(time, 'Day') as weekday,
                      max(date_part('isodow', time)) as weekday_number,
                      count(distinct user_id) as users
               FROM   user_actions
               WHERE  time >= '2022-08-26'
                  and time < '2022-09-09'
               GROUP BY weekday) t2 using (weekday)
    LEFT JOIN (SELECT to_char(time, 'Day') as weekday,
                      max(date_part('isodow', time)) as weekday_number,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
                  and time >= '2022-08-26'
                  and time < '2022-09-09'
               GROUP BY weekday) t3 using (weekday)
ORDER BY weekday_number
```

#### 5. Посчитаем ежедневную выручку с заказов новых пользователей нашего сервиса. Посмотрим, какую долю она составляет в общей выручке с заказов всех пользователей — и новых, и старых.
Таким образом следующие показатели:
* Выручку, полученную в этот день.
* Выручку с заказов новых пользователей, полученную в этот день.
* Долю выручки с заказов новых пользователей в общей выручке, полученной за этот день.
* Долю выручки с заказов остальных пользователей в общей выручке, полученной за этот день.

```sql
SELECT date,
       sum(order_price) as revenue,
       sum(order_price) filter(WHERE date = first_date) as new_users_revenue,
       round(sum(order_price) filter(WHERE date = first_date)::decimal/sum(order_price)*100,
             2) as new_users_revenue_share,
       round(((sum(order_price) - sum(order_price) filter(WHERE date = first_date))::decimal/sum(order_price))*100,
             2) as old_users_revenue_share
FROM   (SELECT creation_time::date as date,
               sum(price) as order_price,
               order_id
        FROM   (SELECT creation_time,
                       unnest(product_ids) as product_id,
                       order_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using (product_id)
        GROUP BY date, order_id) t2
    LEFT JOIN (SELECT user_id,
                      order_id,
                      min(date(time)) OVER(PARTITION BY user_id) as first_date
               FROM   user_actions) t3 using (order_id)
GROUP BY date
ORDER BY date
```

#### 6. Интересно посмотреть, какие товары пользуются наибольшим спросом и приносят основной доход.
Посчитаем:
* Суммарную выручку, полученную от продажи этого товара за весь период.
* Долю выручки от продажи этого товара в общей выручке, полученной за весь период.
Товары, округлённая доля которых в выручке составляет менее 0.5%, объединю в общую группу с названием «ДРУГОЕ» (без кавычек), просуммировав округлённые доли этих товаров.

```sql
SELECT product_name,
       sum(revenue) as revenue,
       sum(share_in_revenue) as share_in_revenue
FROM   (SELECT case when round(100 * revenue / sum(revenue) OVER (), 2) >= 0.5 then name
                    else 'ДРУГОЕ' end as product_name,
               revenue,
               round(100 * revenue / sum(revenue) OVER (), 2) as share_in_revenue
        FROM   (SELECT name,
                       sum(price) as revenue
                FROM   (SELECT order_id,
                               unnest(product_ids) as product_id
                        FROM   orders
                        WHERE  order_id not in (SELECT order_id
                                                FROM   user_actions
                                                WHERE  action = 'cancel_order')) t1
                    LEFT JOIN products using(product_id)
                GROUP BY name) t2) t3
GROUP BY product_name
ORDER BY revenue desc
```

#### 7. Для каждого дня в таблицах orders и courier_actions рассчитаю следующие показатели:

1. Выручку, полученную в этот день.
2. Затраты, образовавшиеся в этот день.
3. Сумму НДС с продажи товаров в этот день.
4. Валовую прибыль в этот день (выручка за вычетом затрат и НДС).
5. Суммарную выручку на текущий день.
6. Суммарные затраты на текущий день.
7. Суммарный НДС на текущий день.
8. Суммарную валовую прибыль на текущий день.
9. Долю валовой прибыли в выручке за этот день.
10. Долю суммарной валовой прибыли в суммарной выручке на текущий день.

В упрощённом виде затраты нашего сервиса будем считать как сумму постоянных и переменных издержек. К постоянным издержкам отнесём аренду складских помещений, а к переменным — стоимость сборки и доставки заказа. Таким образом, переменные затраты будут напрямую зависеть от числа заказов.

Из данных, которые нам предоставил финансовый отдел, известно, что в августе 2022 года постоянные затраты составляли 120 000 рублей в день. Однако уже в сентябре нашему сервису потребовались дополнительные помещения, и поэтому постоянные затраты возросли до 150 000 рублей в день.

Также известно, что в августе 2022 года сборка одного заказа обходилась нам в 140 рублей, при этом курьерам мы платили по 150 рублей за один доставленный заказ и ещё 400 рублей ежедневно в качестве бонуса, если курьер доставлял не менее 5 заказов в день. В сентябре продакт-менеджерам удалось снизить затраты на сборку заказа до 115 рублей, но при этом пришлось повысить бонусную выплату за доставку 5 и более заказов до 500 рублей, чтобы обеспечить более конкурентоспособные условия труда. При этом в сентябре выплата курьерам за один доставленный заказ осталась неизменной.

```sql
SELECT date,
       revenue,
       costs,
       tax,
       gross_profit,
       total_revenue,
       total_costs,
       total_tax,
       total_gross_profit,
       round(gross_profit / revenue * 100, 2) as gross_profit_ratio,
       round(total_gross_profit / total_revenue * 100, 2) as total_gross_profit_ratio
FROM   (SELECT date,
               revenue,
               costs,
               tax,
               revenue - costs - tax as gross_profit,
               sum(revenue) OVER (ORDER BY date) as total_revenue,
               sum(costs) OVER (ORDER BY date) as total_costs,
               sum(tax) OVER (ORDER BY date) as total_tax,
               sum(revenue - costs - tax) OVER (ORDER BY date) as total_gross_profit
        FROM   (SELECT date,
                       orders_packed,
                       orders_delivered,
                       couriers_count,
                       revenue,
                       case when date_part('month', date) = 8 then 120000.0 + 140 * coalesce(orders_packed, 0) + 150 * coalesce      (orders_delivered, 0) + 400 * coalesce(couriers_count, 0)
                            when date_part('month', date) = 9 then 150000.0 + 115 * coalesce(orders_packed, 0) + 150 * coalesce(orders_delivered, 0) + 500 * coalesce(couriers_count, 0) 
                            end as costs, tax
                FROM   (SELECT creation_time::date as date,
                               count(distinct order_id) as orders_packed,
                               sum(price) as revenue,
                               sum(tax) as tax
                        FROM   (SELECT order_id,
                                       creation_time,
                                       product_id,
                                       name,
                                       price,
                                       case when name in ('сахар', 'сухарики', 'сушки', 'семечки', 'масло льняное', 'виноград', 'масло оливковое', 'арбуз', 'батон', 'йогурт', 'сливки', 'гречка', 'овсянка', 'макароны', 'баранина', 'апельсины', 'бублики', 'хлеб', 'горох', 'сметана', 'рыба копченая', 'мука', 'шпроты', 'сосиски', 'свинина', 'рис', 'масло кунжутное', 'сгущенка', 'ананас', 'говядина', 'соль', 'рыба вяленая', 'масло подсолнечное', 'яблоки', 'груши', 'лепешка', 'молоко', 'курица', 'лаваш', 'вафли', 'мандарины') then round(price/110*10, 2)
                                          else round(price/120*20, 2) end as tax
                                FROM   (SELECT order_id,
                                               creation_time,
                                               unnest(product_ids) as product_id
                                        FROM   orders
                                        WHERE  order_id not in (SELECT order_id
                                                                FROM   user_actions
                                                                WHERE  action = 'cancel_order')) t1
                                    LEFT JOIN products using (product_id)) t2
                        GROUP BY date) t3
                    LEFT JOIN (SELECT time::date as date,
                                      count(distinct order_id) as orders_delivered
                               FROM   courier_actions
                               WHERE  order_id not in (SELECT order_id
                                                       FROM   user_actions
                                                       WHERE  action = 'cancel_order')
                                  and action = 'deliver_order'
                               GROUP BY date) t4 using (date)
                    LEFT JOIN (SELECT date,
                                      count(courier_id) as couriers_count
                               FROM   (SELECT time::date as date,
                                              courier_id,
                                              count(distinct order_id) as orders_delivered
                                       FROM   courier_actions
                                       WHERE  order_id not in (SELECT order_id
                                                               FROM   user_actions
                                                               WHERE  action = 'cancel_order')
                                          and action = 'deliver_order'
                                       GROUP BY date, courier_id having count(distinct order_id) >= 5) t5
                               GROUP BY date) t6 using (date)) t7) t8
```

Результаты всех запросов визуализированы в редаш и представлены в описании к проекту